In [9]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [10]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [11]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [12]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_14_9_4,0.992510,0.993984,0.987932,0.990365,0.008289,0.005161,0.023543,0.013812,"Hidden Size=[35], regularizer=0.1, learning_ra..."
1,model_14_9_5,0.992487,0.993874,0.985107,0.988521,0.008315,0.005255,0.029055,0.016455,"Hidden Size=[35], regularizer=0.1, learning_ra..."
2,model_14_9_3,0.992345,0.994031,0.990689,0.992146,0.008471,0.005121,0.018164,0.011259,"Hidden Size=[35], regularizer=0.1, learning_ra..."
3,model_14_9_6,0.992324,0.993720,0.982279,0.986661,0.008495,0.005388,0.034571,0.019121,"Hidden Size=[35], regularizer=0.1, learning_ra..."
4,model_14_9_7,0.992061,0.993534,0.979498,0.984821,0.008786,0.005547,0.039997,0.021759,"Hidden Size=[35], regularizer=0.1, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...
317,model_7_8_24,0.981356,0.954969,0.967996,0.965188,0.020633,0.028014,0.059694,0.042922,"Hidden Size=[20], regularizer=0.01, learning_r..."
319,model_7_8_21,0.981356,0.954969,0.967996,0.965188,0.020633,0.028014,0.059694,0.042922,"Hidden Size=[20], regularizer=0.01, learning_r..."
320,model_7_8_22,0.981356,0.954969,0.967996,0.965188,0.020633,0.028014,0.059694,0.042922,"Hidden Size=[20], regularizer=0.01, learning_r..."
321,model_7_8_0,0.981356,0.954969,0.967996,0.965188,0.020633,0.028014,0.059694,0.042922,"Hidden Size=[20], regularizer=0.01, learning_r..."
